# AlexNet in Keras

In this notebook, we leverage an [AlexNet](https://papers.nips.cc/paper/4824-imagenet-classification-with-deep-convolutional-neural-networks)-like deep, convolutional neural network to classify flowers into the 17 categories of the [Oxford Flowers](http://www.robots.ox.ac.uk/~vgg/data/flowers/17/) data set. Derived from [this earlier notebook](https://github.com/the-deep-learners/TensorFlow-LiveLessons/blob/master/notebooks/old/L3-3b__TFLearn_AlexNet.ipynb).

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/the-deep-learners/deep-learning-illustrated/blob/master/notebooks/alexnet_in_keras.ipynb)

#### Load dependencies

In [1]:
!pip install --quiet tensorflow matplotlib

In [8]:
import os
import shutil

# 確保來源資料夾正確
print("圖片來源：", data_dir)
print("圖片範例：", os.listdir(data_dir)[:5])

# 強制重建分類資料夾
base_dir = "/tmp/oxford17_split"
if os.path.exists(base_dir):
    shutil.rmtree(base_dir)  # 🔁 清空舊資料
os.makedirs(base_dir)

# 每類 80 張，共 17 類
for class_id in range(17):
    class_dir = os.path.join(base_dir, f"class_{class_id}")
    os.makedirs(class_dir)
    for img_id in range(class_id * 80 + 1, (class_id + 1) * 80 + 1):
        src = os.path.join(data_dir, f"image_{img_id:04d}.jpg")
        dst = os.path.join(class_dir, f"image_{img_id:04d}.jpg")
        if os.path.exists(src):
            shutil.copy(src, dst)
        else:
            print(f"❗ 找不到圖片：{src}")



圖片來源： /root/.keras/datasets/oxford17/jpg
圖片範例： ['image_0366.jpg', 'image_1251.jpg', 'image_0617.jpg', 'image_0190.jpg', 'image_0091.jpg']


In [9]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    base_dir, validation_split=0.1, subset="training", seed=123,
    image_size=(224, 224), batch_size=32
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    base_dir, validation_split=0.1, subset="validation", seed=123,
    image_size=(224, 224), batch_size=32
)


Found 1360 files belonging to 17 classes.
Using 1224 files for training.
Found 1360 files belonging to 17 classes.
Using 136 files for validation.


#### Load *and preprocess* data

#### Design neural network architecture

In [10]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from keras import Input

model = Sequential()
model.add(Input(shape=(224, 224, 3)))

model.add(Conv2D(96, kernel_size=(11, 11), strides=(4, 4), activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
model.add(BatchNormalization())

model.add(Conv2D(256, kernel_size=(5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
model.add(BatchNormalization())

model.add(Conv2D(256, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(384, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(384, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
model.add(BatchNormalization())

model.add(Flatten())
model.add(Dense(4096, activation='tanh'))
model.add(Dropout(0.5))
model.add(Dense(4096, activation='tanh'))
model.add(Dropout(0.5))
model.add(Dense(17, activation='softmax'))

In [11]:
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 54, 54, 96)     │        34,944 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 26, 26, 96)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 26, 26, 96)     │           384 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 22, 22, 256)    │       614,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 10, 10, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 10, 10, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 8, 8, 256)      │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 6, 6, 384)      │       885,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 4, 4, 384)      │     1,327,488 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 1, 1, 384)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 1, 1, 384)      │         1,536 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 384)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 4096)           │     1,576,960 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4096)           │    16,781,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 17)             │        69,649 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 21,883,153 (83.48 MB)

 Trainable params: 21,881,681 (83.47 MB)

 Non-trainable params: 1,472 (5.75 KB)

#### Configure model

In [12]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

#### Train!

In [13]:
model.fit(train_ds, validation_data=val_ds, epochs=10)

Epoch 1/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 116s 3s/step - accuracy: 0.1539 - loss: 4.4878 - val_accuracy: 0.0441 - val_loss: 5.1346
Epoch 2/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 107s 3s/step - accuracy: 0.1692 - loss: 3.5340 - val_accuracy: 0.0221 - val_loss: 5.1970
Epoch 3/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 142s 3s/step - accuracy: 0.1779 - loss: 3.6065 - val_accuracy: 0.1324 - val_loss: 4.1367
Epoch 4/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 143s 3s/step - accuracy: 0.2803 - loss: 2.9774 - val_accuracy: 0.0368 - val_loss: 9.0643
Epoch 5/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 143s 3s/step - accuracy: 0.2618 - loss: 3.0858 - val_accuracy: 0.2279 - val_loss: 4.0180
Epoch 6/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 142s 3s/step - accuracy: 0.3184 - loss: 2.7836 - val_accuracy: 0.2059 - val_loss: 3.4475
Epoch 7/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 140s 3s/step - accuracy: 0.3419 - loss: 2.5097 - val_accuracy: 0.2721 - val_loss: 3.6215
Epoch 8/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 145s 3s/step - accuracy: 0.3489 - loss: 2.8372 - val_accuracy: 0.1176 - v